<a href="https://colab.research.google.com/github/diptisharnagat/NLP/blob/main/NLP_EXP_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


Sample data (context and senses)

In [2]:
data = [
    (["The", "bank", "by", "the", "river", "is", "steep."], "financial_institution"),
    (["I", "walked", "along", "the", "river", "bank", "yesterday."], "river_bank"),
]

Create a vocabulary

In [3]:
vocab = set(word for context, _ in data for word in context)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

Map sense labels to integers

In [4]:
sense_labels = list(set(label for _, label in data))
sense_to_idx = {sense: idx for idx, sense in enumerate(sense_labels)}
idx_to_sense = {idx: sense for sense, idx in sense_to_idx.items()}

Convert data to tensors

In [5]:
data_tensors = [(torch.tensor([word_to_idx[word] for word in context]), torch.tensor(sense_to_idx[sense])) for context, sense in data]

Define the LSTM-based WSD model


In [6]:
class WSDModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, sense_count):
        super(WSDModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, sense_count)

    def forward(self, context):
        embedded = self.embedding(context)
        lstm_out, _ = self.lstm(embedded.view(len(context), 1, -1))
        prediction = self.fc(lstm_out[-1])
        return prediction


Hyperparameters

In [7]:
vocab_size = len(vocab)
embedding_dim = 100
hidden_dim = 64
sense_count = len(sense_labels)
learning_rate = 0.001
epochs = 10

Initialize the model

In [8]:
model = WSDModel(vocab_size, embedding_dim, hidden_dim, sense_count)


Define the loss function and optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Training loop

In [11]:
def train(model, data, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for context, target_sense in data:
            optimizer.zero_grad()
            output = model(context)
            loss = criterion(output, target_sense.unsqueeze(0))  # Add batch dimension to target
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(data)}")


Train the model

In [12]:
train(model, data_tensors, criterion, optimizer, epochs)

Epoch 1/10, Loss: 0.7736735045909882
Epoch 2/10, Loss: 0.6583276093006134
Epoch 3/10, Loss: 0.5681799650192261
Epoch 4/10, Loss: 0.48926404118537903
Epoch 5/10, Loss: 0.41954848170280457
Epoch 6/10, Loss: 0.35778510570526123
Epoch 7/10, Loss: 0.3031216561794281
Epoch 8/10, Loss: 0.25498469918966293
Epoch 9/10, Loss: 0.21295592933893204
Epoch 10/10, Loss: 0.1766686663031578


Inference (predict senses for new contexts)


In [13]:
with torch.no_grad():
    new_context = ["The", "bank", "charges", "high", "fees."]
    new_context = torch.tensor([word_to_idx.get(word, 0) for word in new_context])
    new_context = new_context.unsqueeze(0)  # Add batch dimension
    predictions = model(new_context)
    predicted_label = idx_to_sense[torch.argmax(predictions).item()]
    print(f"Predicted sense: {predicted_label}")

Predicted sense: river_bank
